Project by Jeremy Bouhi & Lucas Trevalinet

### Classification Document


In [1]:
# Imports

import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path

In [2]:
# STEP 1 : GRAB DATA
# We consider you had already unzip the dataset file and that this file is in the parent folder

GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global")

CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

# We need to do that for excluding data from Juri_path
DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

In [7]:
import re
ident = []
text = []
division = []
DATA_PATH = [CIVILE_PATH]

for DIVISION_PATH in DATA_PATH :
    #xml_files = list(Path(DIVISION_PATH).glob('**/*.xml')) En vrai c'est ça, mais pour tester, autant prendre qu'un seul dossier
    xml_files = list(Path(CIVILE_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            #print (root)
            
            
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)
                
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = re.sub('<[^<]+?>', '', ET.tostring(child).decode("utf-8"))
                text.append(contenu)
                

            # For getting the text
            #for child in root.iter('BLOC_TEXTUEL'):
             #   contenu = child.find('CONTENU').text
              #  print (contenu)
               # text.append(contenu)
            
                

            # For getting the division
            for child in root.iter('META_JURI_JUDI'):
                formation = child.find('FORMATION').text
                division.append(formation)

In [8]:
print(text[0])


ANNULATION, sur la demande du sieur Prosper X..., avou&#233;, d'un arr&#234;t rendu, par la Cour royale de Rouen, le 18 f&#233;vrier 1842, au profit du sieur Y... et autres.

                         Du 30 juillet 1844.
                           NOTICE ET MOTIFS.
          Le 13 avril 1838, le sieur Y..., avou&#233; pr&#232;s la cour royale de Rouen, c&#233;da son office au sieur X..., avec ses recouvrements et quelques objets mobiliers.
          Mais deux trait&#233;s s&#233;par&#233;s furent souscrits. Par l'un, destin&#233; &#224; &#234;tre soumis &#224; l'autorit&#233;, le prix de l'office &#233;tait port&#233; &#224; 85000 francs seulement, payables &#224; diverses &#233;poques. Le second trait&#233;, qui devait rester secret entre les parties, stipulait un exc&#233;dent de prix d'une somme de 31500 francs, payable la veille de la prestation de serment du nouveau titulaire.
         Le sieur X... fut en effet nomm&#233; avou&#233; en remplacement du sieur Y....         
       

In [9]:
print(ident[59249])

JURITEXT000036697099


In [10]:
print ("text: ", len(text))
print ("ID: ", len(ident))
print ("division: ", len(division))

text:  59250
ID:  59250
division:  59250


In [11]:
# STEP 2 : CREATE DATAFRAME TO MANIPULATE EASILY DATA

d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)

df
#df.loc[df['id'] == 'JURITEXT000006951557']

,id,text,division
0,JURITEXT000006951557,"\nANNULATION, sur la demande du sieur Prosper ...",CHAMBRE_CIVILE
1,JURITEXT000006951558,\nREJET du pourvoi form&#233; par le sieur Y.....,CHAMBRE_CIVILE
2,JURITEXT000006951610,"\nANNULATION, sur la demande des sieurs G... d...",CHAMBRE_CIVILE
3,JURITEXT000006951611,"\nANNULATION, sur la demande des h&#233;ritier...",CHAMBRE_CIVILE
4,JURITEXT000006951612,"\n ANNULATION, sur la demande de M. le...",CHAMBRE_CIVILE
5,JURITEXT000006951614,"\nANNULATION, sur le pourvoi form&#233; par le...",CHAMBRE_CIVILE
6,JURITEXT000006951615,"\n\nANNUALTION, sur le pourvoi du sieur Y..., ...",CHAMBRE_CIVILE
7,JURITEXT000006951616,"\nANNULATION, sur le pourvoi du sieur Z... et ...",CHAMBRE_CIVILE
8,JURITEXT000006951618,\nREJET du pourvoi du sieur Y... contre un Arr...,CHAMBRE_CIVILE
9,JURITEXT000006951664,\n\n\n,CHAMBRE_CIVILE_1


In [8]:
# STEP 3 : CLEAN DATA
